In [1]:
pip install jmespath parsel unidecode pyodbc

     ------------------------------------ 235.5/235.5 kB 801.6 kB/s eta 0:00:00
     ---------------------------------------- 3.8/3.8 MB 7.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import json
import requests
import jmespath
from parsel import Selector


In [ ]:
def SaveJSON(fileName,JsonComic):
  print(fileName+"\n")
  with open(f"{fileName}.json", "w") as outfile:
      json.dump(JsonComic, outfile,indent= 2)

In [21]:
f = open('mapComic.json')
ListComic = json.load(f)

In [38]:
allcomicstring ="""
        {
          "comic": props.pageProps.dataComicNew[*].
          {
            urlComic:urlComic,
            totalChapter:totalChapter,
            createDateComic:createDateComic,
            lastUpdateChapter:lastUpdateChapter
          }
        }
"""
def GetAllComicMapJSON():
  ListComic = {"comic":[]}
  expression = jmespath.compile( allcomicstring )
  for idx in range(1, 600):
    response = requests.get(f'https://doctruyenonline.vn/truyen-tranh/tim-truyen?status=2&sort=2&page={idx}')
    if(response.status_code != 200):
      print(f'Fetching Error Page{idx} {response.status_code}')
      continue
    print(f'Fetching Page{idx} {response.status_code}')
    selector = Selector(text=response.text)
    json_data = selector.css("script[type='application/json']::text").get()
    data = json.loads(json_data)
    flat_data = expression.search(data)
    if(flat_data["comic"] == []): break
    ListComic["comic"].extend(flat_data["comic"])
  return ListComic


In [ ]:
ListComic = GetAllComicMapJSON()

In [50]:
page_not_found = []
comic_not_found = []

In [53]:
def GetComicJSON(urlComic):
  global comic_not_found
  response = requests.get(f'https://doctruyenonline.vn/truyen-tranh/{urlComic}')
  if (response.status_code != 200):
    print(response.status_code)
    comic_not_found.append(urlComic)
    return None
  selector = Selector(text=response.text)
  json_data = selector.css("script[type='application/json']::text").get()

  data = json.loads(json_data)
  expression = jmespath.compile(
      """
      {
        "Title": props.pageProps.comic[0].nameComic,
        "URL": props.pageProps.comic[0].urlComic,
        "CoverImage":   props.pageProps.comic[0].imgComic,
        "CreateAt":  props.pageProps.comic[0].create_date
        "statusComic": props.pageProps.comic[0].statusComic
        "Category":   props.pageProps.comic[0].category[*].name_cate,
        "LastChapter": props.pageProps.comic[0].lastChapter
        "TotalChapter": props.pageProps.comic[0].totalChapter
        "Chapters": props.pageProps.listChapter[0].{urlChapter:urlChapter, dateUpdateChapter: dateUpdateChapter}
      }
      """
  )
  flat_data = expression.search(data)
  return flat_data


def fetchData(start = 0, end = 100):
    global ListComic
    for idx,oneComic in  enumerate(ListComic["comic"][start:end]):
        urlComic = oneComic["urlComic"]
        JsonComic = GetComicJSON(urlComic)
        if (JsonComic == None): continue
        print(f"({idx},{JsonComic['Title']})")
        SaveJSON(f"ComicData/{urlComic}",JsonComic)

In [60]:
for i in range(100,1100,100):
    print(i)

100
200
300
400
500
600
700
800
900
1000


In [ ]:
import threading
for i in range(100,10000,100):
    th1 = threading.Thread(target=fetchData, args=(i, i+100))
    th1.start()

In [31]:
SaveJSON(f"mapComic",ListComic)

mapComic



In [30]:

# th1 = threading.Thread(target=fetchData, args=(6, 20))
# th2 = threading.Thread(target=fetchData, args=(21, 40))
# th3 = threading.Thread(target=fetchData, args=(41, 60))
# th4 = threading.Thread(target=fetchData, args=(61, 80))
# th5 = threading.Thread(target=fetchData, args=(81, 100))
# th1 = threading.Thread(target=fetchData, args=(101, 120))
# th2 = threading.Thread(target=fetchData, args=(121, 140))
# th3 = threading.Thread(target=fetchData, args=(141, 160))
# th4 = threading.Thread(target=fetchData, args=(161, 180))
# th5 = threading.Thread(target=fetchData, args=(181, 200))
# th1 = threading.Thread(target=fetchData, args=(201, 220))
# th2 = threading.Thread(target=fetchData, args=(221, 240))
# th3 = threading.Thread(target=fetchData, args=(241, 260))
# th4 = threading.Thread(target=fetchData, args=(261, 280))
# th5 = threading.Thread(target=fetchData, args=(281, 300))
th1 = threading.Thread(target=fetchData, args=(301, 340))
th2 = threading.Thread(target=fetchData, args=(341, 380))
th3 = threading.Thread(target=fetchData, args=(381, 420))
th4 = threading.Thread(target=fetchData, args=(421, 460))
th5 = threading.Thread(target=fetchData, args=(461, 500))


th1.start()
th2.start()
th3.start()
th4.start()
th5.start()

th1.join()
th2.join()
th3.join()
th4.join()
th5.join()


fetching page 301

fetching page 341

fetching page 381

fetching page 421

fetching page 461

status 200
status 200
status 200
status 200
status 200
ComicData/gen-tho-san

ComicData/16-tuoi-khong-mot-nguoi-theo-duoi

ComicData/dung-tuy-tien-chong-lai-cap-tren

ComicData/khuat-phuc-anh-chong-phan-dien-cua-toi

ComicData/thi-than

ComicData/raisekamika

ComicData/thien-nien-tinh-duyen-cong-tu-thinh-lanh-tinh

ComicData/she-was-actually-my-stepsister-recently-the-sense-of-distance-between-me-and-my-new-stepbrother-is-incredibly-close

ComicData/i-m-a-middle-schooler-becoming-the-demonlord

ComicData/katakoto-no-niwa

ComicData/a-bias-girl

ComicData/ban-gai-da-nhan-cach

ComicData/tram-long

ComicData/punisher-vao-vung-chien-su-enter-the-war-zone

ComicData/inuwashi-momo-wa-yuruganai

ComicData/yeu-linh-cuu-hoa

ComicData/manshuu-ahen-squad

ComicData/shinken-de-watashi-ni-koi-shinasai

ComicData/story-about-ol-san-picking-up-a-cat

ComicData/chung-cu-tinh-yeu

ComicData/re-creators

Com

Exception in thread Thread-21 (fetchData):
Traceback (most recent call last):
  File "c:\Users\gihot\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Users\gihot\AppData\Local\Programs\Python\Python310\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\gihot\AppData\Local\Temp\ipykernel_22952\2755586689.py", line 58, in fetchData
TypeError: 'NoneType' object is not iterable


status 200
ComicData/vita-sexualis

ComicData/love-live-dj-nicomaki-medical-check

ComicData/koi-o-kataranai-kuchibiru

ComicData/siscon-onee-chan-to-ki-ni-shinai-imouto

ComicData/lueduo-diren-de-xin

fetching page 383

ComicData/ke-bat-tu

status 200
ComicData/phap-su-muon-song-binh-yen-mot-ngay

ComicData/love-dream-mix

ComicData/rong-ngang-dau

ComicData/yamada-to-sensei

ComicData/huyen-dan-chi-kiem

ComicData/a-city-called-nowhere

500
ComicData/angry-birds

ComicData/tieu-y-thu-cua-tuong-quan

ComicData/tebukuro-wo-shita-onna-no-ko

ComicData/umareru-kachi-no-nakatta-jibun-ga-anna-no-tame-ni-dekiru-ikutsuka-no-koto

ComicData/gimmick

ComicData/shikabane-gatana

ComicData/hakase-to-slime-chan

ComicData/song-chung-la-de-hoc-tap

ComicData/bi-ep-thanh-chua-cuu-the

ComicData/sawaru-no-kinshi-ore-no-mono

ComicData/c3-cube-x-cursed-x-curious

ComicData/sisterism

ComicData/to-tinh-voi-co-ban-thuo-nho-luon-nghi-minh-nhat-nheo

ComicData/toi-da-giet-han

ComicData/mot-ngay-dau-nam-

In [ ]:
fetchData(1, 5)

In [ ]:
GetChapterOfComicJSON("cuop-dau", chapter=1)

In [9]:
def GetChapterOfComicJSON(urlComic, chapter=1):
  response = requests.get(f'https://doctruyenonline.vn/truyen-tranh/{urlComic}/chapter-{chapter}')
  print(response.status_code)
  
  selector = Selector(text=response.text)
  json_data = selector.css("script[type='application/json']::text").get()
  data = json.loads(json_data)
  expression = jmespath.compile(
      """
      {
        "Page": props.pageProps.dataContentChapter[*].{indexImg:indexImg,imgChapter:imgChapter}
      }
      """
  )
  flat_data = expression.search(data)
  return flat_data

In [10]:
JsonComic = GetComicJSON("toi-se-bi-tru-khu-cung-hoang-de")

In [ ]:
JsonComic

In [ ]:
# encodedUnicode = json.dumps(JsonComic, ensure_ascii=False)


In [ ]:
import unidecode
import re
def CreateSlug(accented_string:str):

  # accented_string is of type 'unicode'
  unaccented_string = unidecode.unidecode(accented_string).lower()
  unaccented_string = re.sub(r"[^\w\s]", '', unaccented_string)
  unaccented_string=re.sub(r"\s+","-",unaccented_string)
  return unaccented_string

CreateSlug("Tu Ti\u00ean \u1ede Th\u1ebf Gi\u1edbi Si\u00eau N\u0103ng L\u1ef1c")

In [ ]:
from bs4 import BeautifulSoup

# Assuming we have an HTML document in the 'html_content' variable
soup = BeautifulSoup(html_content, 'html.parser')

# Find the desired element and extract its text using get_text()
elements = soup.find_all('span',itemprop="name")

#get_text with strip set to true
for idx,element in  enumerate(elements):
  print(f"({idx},{element.text})")

In [1]:
import pyodbc 
server = 'DESKTOP-33SUFGJ\\SQLEXPRESS'
database = 'ComicApp'
username = 'your_username'
password = 'your_password'
connectionString = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};'

In [20]:
conn = pyodbc.connect(connectionString)
cursor = conn.cursor()

In [3]:
SQL_QUERY = """
SELECT * FROM COMIC
"""
cursor = conn.cursor()
cursor.execute(SQL_QUERY)


In [4]:
for i in cursor:
    print(i)

In [8]:
import glob

listPath = glob.glob("ComicData/*")
len(listPath)

13800

In [28]:
listPath[41]

'ComicData\\12-nights.json'

In [29]:
SQL_QUERY = """
  SET DATEFORMAT dmy;
  INSERT INTO COMIC (Title,URL,CoverImage,Status,CreateAt,Rating) VALUES
  (?,?,?,?,?,?)
"""


for path in listPath[41:]:
  f = open(path)
  datajson = json.load(f)
  if(not datajson['URL']):
    print(path)
    continue
  VALUES =(datajson['Title'],datajson['URL'],datajson['CoverImage'],datajson['statusComic'],datajson['CreateAt'],10)
  cursor.execute(SQL_QUERY,VALUES)
  conn.commit()
cursor.close()


ComicData\12-nights.json
ComicData\14-sai.json
ComicData\5-tsu-no-hajimete-ubawarete-mo-ii-kimi-ni-nara.json
ComicData\ac-nu-trung-sinh.json
ComicData\ac-quy-ngay-tho.json
ComicData\adventure-and-trouble-diary.json
ComicData\ai-kiss-idol-kiss.json
ComicData\akumu-koushounin.json
ComicData\amakara-gishimai-ni-hasamarete-masu.json
ComicData\ame-no-gogo-wa-romance-no-heroine.json
ComicData\anh-hung-manh-nhat-lich-su-chuyen-sinh-va-vo-tinh-tro-nen-vo-dich-o-hoc-vien.json
ComicData\anti-venom-new-way-to-live.json
ComicData\ban-kiem-tien-tuyet-khong-lam-no.json
ComicData\bara-no-tame-ni.json
ComicData\benh-vien-ma-am.json
ComicData\bien-mua-he.json
ComicData\blood-alone.json
ComicData\boys-next-door.json
ComicData\captain-kid-hoang-tu-cuop-bien.json
ComicData\chap-kiem-gia.json
ComicData\chigihagu-planet.json
ComicData\chim-yen-khong-ve.json
ComicData\chinh-phuc-vo-lam-chi-voi-1-to-my.json
ComicData\chuyen-doi-gioi-tinh.json
ComicData\chuyen-sinh-thanh-kiem.json
ComicData\chuyen-vien-chuyen-

In [25]:
  conn.commit()

In [14]:
cursor.execute(query)

In [12]:
print(query)


  SET DATEFORMAT dmy;
  INSERT INTO COMIC (Title,URL,CoverImage,Status,CreateAt,UpdateAt,Rating) VALUES
  (N'0 no soukoushi',N'0-no-soukoushi','0-no-soukoushi.jpg',1,'31/01/2024',GETDATE(),10)



In [18]:
for i in cursor:
    print(i)

ProgrammingError: Attempt to use a closed cursor.

In [55]:
print(query )


  SET DATEFORMAT dmy;
  INSERT INTO COMIC (Title,URL,CoverImage,Status,CreateAt) VALUES
  (0 no soukoushi,0-no-soukoushi,0-no-soukoushi.jpg,1,31/01/2024)

